# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/Micha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Micha/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [18]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Use-Case Data!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 64, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '907b38'. Skipping!
Property 'summary' already exists in node '373c6c'. Skipping!
Property 'summary' already exists in node '4070b3'. Skipping!
Property 'summary' already exists in node '6e9259'. Skipping!
Property 'summary' already exists in node '8a50a5'. Skipping!
Property 'summary' already exists in node '1b2056'. Skipping!
Property 'summary' already exists in node 'c689b1'. Skipping!
Property 'summary' already exists in node '40ee84'. Skipping!
Property 'summary' already exists in node '6d3e55'. Skipping!
Property 'summary' already exists in node '1b3053'. Skipping!
Property 'summary' already exists in node '83e288'. Skipping!
Property 'summary' already exists in node 'c17daa'. Skipping!
Property 'summary' already exists in node 'c2498b'. Skipping!
Property 'summary' already exists in node '01863a'. Skipping!
Property 'summary' already exists in node '6554ad'. Skipping!
Property 'summary' already exists in node '1af5a3'. Skipping!
Property

Applying CustomNodeFilter:   0%|          | 0/8 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/48 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '6e9259'. Skipping!
Property 'summary_embedding' already exists in node '8a50a5'. Skipping!
Property 'summary_embedding' already exists in node '373c6c'. Skipping!
Property 'summary_embedding' already exists in node '1b2056'. Skipping!
Property 'summary_embedding' already exists in node 'c17daa'. Skipping!
Property 'summary_embedding' already exists in node '907b38'. Skipping!
Property 'summary_embedding' already exists in node '83e288'. Skipping!
Property 'summary_embedding' already exists in node '4070b3'. Skipping!
Property 'summary_embedding' already exists in node '6d3e55'. Skipping!
Property 'summary_embedding' already exists in node '40ee84'. Skipping!
Property 'summary_embedding' already exists in node 'c689b1'. Skipping!
Property 'summary_embedding' already exists in node '1b3053'. Skipping!
Property 'summary_embedding' already exists in node 'c2498b'. Skipping!
Property 'summary_embedding' already exists in node '01863a'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 86, relationships: 711)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 86, relationships: 711)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

#### ✅ Answer #1:

**SingleHopSpecificQuerySynthesizer**
Generates simple questions that can be answered with a single piece of information from one source (simple facts).
*Example:* What is the capital of Paris 😉

**MultiHopAbstractQuerySynthesizer**
Generates abstract questions that need complex reasoning (connecting concepts)
*Example* How do neural networks relate to artificial intelligence?

**MultiHopSpecificQuerySynthesizer**
Generates questions that need multiple specific facts to answer (complex facts)
*Example* What are the advantages of deep learning over traditional machine learning?

Finally, we can use our `TestSetGenerator` to generate our testset!

In [13]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,"What does Acemoglu, 2024 say about AI's impact?",[Introduction ChatGPT launched in November 202...,"The text mentions that Acemoglu, 2024, is one ...",single_hop_specifc_query_synthesizer
1,How does the AI model Claude compare to ChatGP...,[Table 1: ChatGPT daily message counts (millio...,"According to the provided context, ChatGPT's d...",single_hop_specifc_query_synthesizer
2,Computer related occupations use ChatGPT?,[Variation by Occupation Figure 23 presents va...,Variation by Occupation Figure 23 shows that 5...,single_hop_specifc_query_synthesizer
3,What role does personal reflection play in und...,[Conclusion This paper studies the rapid growt...,The provided context does not mention personal...,single_hop_specifc_query_synthesizer
4,How does the prvacy-preserving methdology in C...,[<1-hop>\n\nConclusion This paper studies the ...,The paper introduces a novel privacy-preservin...,multi_hop_abstract_query_synthesizer
5,how much of chatgpt messages are non work and ...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,"In June 2024, non-work messages accounted for ...",multi_hop_abstract_query_synthesizer
6,Based on the data showing that the most common...,[<1-hop>\n\nConclusion This paper studies the ...,The context indicates that Writing is the most...,multi_hop_abstract_query_synthesizer
7,Considering the rapid growth of ChatGPT usage ...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,"The context indicates that in the US, over 70%...",multi_hop_specific_query_synthesizer
8,"Wha US is the US that ChatGPT use in, and how ...",[<1-hop>\n\nTable 1: ChatGPT daily message cou...,The context indicates that the US refers to th...,multi_hop_specific_query_synthesizer
9,How does the rapid growth of ChatGPT to over 7...,[<1-hop>\n\nConclusion This paper studies the ...,"By July 2025, ChatGPT had been used weekly by ...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '5f62f9'. Skipping!
Property 'summary' already exists in node '9e619d'. Skipping!
Property 'summary' already exists in node '585552'. Skipping!
Property 'summary' already exists in node '75a48b'. Skipping!
Property 'summary' already exists in node '3fc6fa'. Skipping!
Property 'summary' already exists in node '7a85bf'. Skipping!
Property 'summary' already exists in node 'f1a0dd'. Skipping!
Property 'summary' already exists in node 'e5c7ad'. Skipping!
Property 'summary' already exists in node '38c096'. Skipping!
Property 'summary' already exists in node '2ea688'. Skipping!
Property 'summary' already exists in node '02cb4d'. Skipping!
Property 'summary' already exists in node 'f4b3b0'. Skipping!
Property 'summary' already exists in node 'e8b8c5'. Skipping!
Property 'summary' already exists in node 'c24658'. Skipping!
Property 'summary' already exists in node '823495'. Skipping!
Property 'summary' already exists in node '3d4517'. Skipping!
Property

...

Property 'summary_embedding' already exists in node '9e619d'. Skipping!
Property 'summary_embedding' already exists in node 'f1a0dd'. Skipping!
Property 'summary_embedding' already exists in node 'e5c7ad'. Skipping!
Property 'summary_embedding' already exists in node '38c096'. Skipping!
Property 'summary_embedding' already exists in node '3fc6fa'. Skipping!
Property 'summary_embedding' already exists in node '5f62f9'. Skipping!
Property 'summary_embedding' already exists in node '585552'. Skipping!
Property 'summary_embedding' already exists in node '7a85bf'. Skipping!
Property 'summary_embedding' already exists in node '75a48b'. Skipping!
Property 'summary_embedding' already exists in node '02cb4d'. Skipping!
Property 'summary_embedding' already exists in node '2ea688'. Skipping!
Property 'summary_embedding' already exists in node 'e8b8c5'. Skipping!
Property 'summary_embedding' already exists in node 'f4b3b0'. Skipping!
Property 'summary_embedding' already exists in node 'c24658'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"Kulveit et al., 2025 what they say about AI ch...",[Introduction ChatGPT launched in November 202...,"The context mentions Kulveit et al., 2025 in r...",single_hop_specifc_query_synthesizer
1,How does OpenAI's development influence user i...,[Table 1: ChatGPT daily message counts (millio...,The context discusses that nearly 80% of all C...,single_hop_specifc_query_synthesizer
2,Could you explain the significance of SOC2 cod...,[Variation by Occupation Figure 23 presents va...,Variation by Occupation Figure 23 presents var...,single_hop_specifc_query_synthesizer
3,What is the significance of Personal Reflectio...,[Conclusion This paper studies the rapid growt...,"According to the context, Personal Reflection ...",single_hop_specifc_query_synthesizer
4,Considering the rapid growth of ChatGPT's usag...,[<1-hop>\n\nConclusion This paper studies the ...,The data indicates that a significant portion ...,multi_hop_abstract_query_synthesizer
5,How does the increasing percentage of non-work...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,The data shows that non-work messages increase...,multi_hop_abstract_query_synthesizer
6,How do usage patterns and growth across popula...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,The context indicates that ChatGPT's adoption ...,multi_hop_abstract_query_synthesizer
7,Considering the rapid growth of ChatGPT's usag...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"The rapid adoption of ChatGPT, with over 18 bi...",multi_hop_abstract_query_synthesizer
8,"Based on the findings of Handa et al. (2025), ...",[<1-hop>\n\nTable 1: ChatGPT daily message cou...,Handa et al. (2025) report that nearly 80% of ...,multi_hop_specific_query_synthesizer
9,what Handa et al 2025 say about ChatGPT messag...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,Handa et al. (2025) report that nearly 80% of ...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [21]:
from langsmith import Client

client = Client()

dataset_name = "Use Case Synthetic Data - AIE8"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [22]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [23]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [25]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [26]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG"
)

In [27]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [28]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [30]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [31]:
rag_chain.invoke({"question" : "What are people doing with AI these days?"})

'Based on the provided context, people are using AI, particularly generative AI like ChatGPT, in a variety of ways including:\n\n- Performing workplace tasks, either augmenting or automating human labor.\n- Producing writing, software code, spreadsheets, and other digital products.\n- Seeking information and advice, similar to traditional web search engines but with more flexibility.\n- Using AI as co-workers producing output or as co-pilots giving advice and improving human problem-solving productivity.\n- Engaging in activities related to relationships, personal reflection, games, and role play, though these are a smaller share of usage.\n- Therapy and companionship are also noted as prevalent use cases for generative AI.\n\nOverall, generative AI is highly flexible and is used both for professional purposes such as labor and decision-making, and for personal purposes such as self-expression and companionship.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [32]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [33]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this response dope, lit, cool, or is it just a generic response?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: evaluates the quality of question-answer pairs -> "How well is our RAG system answering questions" It checks
  - **The answer quality**: How good is the generated answer.
  - **Relevance**: Does the answer actually address the question?
  - **Completeness**: Is the answer complete and informative?
  - **Accuracy**: Is the answer factually correct?

- `labeled_helpfulness_evaluator`: evaluates if the answer is helpful to the user.
- `dopeness_evaluator`: evaluates how "dope" or "cool" the answer is

## LangSmith Evaluation

In [34]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'fresh-detail-16' at:
https://smith.langchain.com/o/54cd3916-cf32-4735-9053-ad02339e1abb/datasets/d9f59057-5dde-4610-b0b5-85fc4c9f7549/compare?selectedSessions=2f0bbd8a-d506-42ee-ab94-38a8c6171787




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"what July 2025 data show ChatGPT use like, and...","By July 2025, ChatGPT had more than 700 millio...",None,"In July 2025, ChatGPT had over 700 million wee...",1,0,0,8.497519,747940ab-46fb-44cd-bae8-27ccf2ff0875,57f50fab-e332-4c06-b4d0-ee85c5508ea3
1,Based on the data indicating that over 700 mil...,"Based on the provided context, the widespread ...",None,"The context shows that by July 2025, ChatGPT h...",1,1,0,4.725201,89fb0173-2ce7-42ad-b075-4a055a5b34ff,1c3f5d67-7b12-45b9-a438-13817b0a9289
2,what Handa et al 2025 say about ChatGPT messag...,Handa et al. (2025) are mentioned in the conte...,None,Handa et al. (2025) report that nearly 80% of ...,0,0,0,1.804315,e76d8c70-f9a1-4088-81e8-45c2c31b9156,16d28117-bcb5-4f30-afd5-7a8979cf4ffa
3,"Based on the findings of Handa et al. (2025), ...","Based on the context provided, the 2025 study ...",None,Handa et al. (2025) report that nearly 80% of ...,1,1,0,5.289896,24f60c41-7ee4-487f-b6de-24977c773987,22932ffd-3918-4c50-87b7-0929657b82cf
4,Considering the rapid growth of ChatGPT's usag...,"Based on the provided context, the analysis us...",None,"The rapid adoption of ChatGPT, with over 18 bi...",1,0,0,3.741153,e92463db-b06c-434c-86fa-79d6eff672ba,5c952963-89c3-4878-bf75-10c52575636a
5,How do usage patterns and growth across popula...,"Based on the provided context, usage patterns ...",None,The context indicates that ChatGPT's adoption ...,1,0,0,11.417308,de7dcde9-f91f-408f-bc2a-4446a1cf85c0,0793737d-f274-4045-be82-96e38504f534
6,How does the increasing percentage of non-work...,"Based on the provided context, the proportion ...",None,The data shows that non-work messages increase...,1,0,0,6.644545,87e21c66-c293-453d-9ff6-a567b3b32e73,085174bc-ed5c-4d9f-8074-724c118e31f6
7,Considering the rapid growth of ChatGPT's usag...,I don't know.,None,The data indicates that a significant portion ...,0,0,0,0.806713,71939f95-13fe-4731-8c04-1c9c8a2ca6b7,80679a06-eb19-44b7-89f6-620eaffd9409
8,What is the significance of Personal Reflectio...,"Based on the provided context, Personal Reflec...",None,"According to the context, Personal Reflection ...",1,1,0,3.090913,2118fe57-7aee-4432-8fb3-06335972010b,947e3e30-698b-4eaa-9346-6e30a832bcd0
9,Could you explain the significance of SOC2 cod...,Based on the provided context:\n\nSOC2 code 13...,None,Variation by Occupation Figure 23 presents var...,1,1,0,7.577661,650a4f17-8cb9-40b1-b9b0-c3af9dc2eeb0,aa4708da-c0c4-4ab7-97ca-11dbd3906370


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [35]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [36]:
rag_documents = docs

In [44]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓ Question #2:

Why would modifying our chunk size modify the performance of our application?

#### ✅ Answer #2:

By changing the chunk size (in our case: making it bigger), the amount of information per chunk changes. Smaller chunks have more focused information, but relationships between information or their 'context' might get lost. Bigger chunks can preserve relationships between information better, but can also include irrelevant information that dilutes the context.

We have 50% single-hop questions and 50% multi-hop questions. Single-hop questions might perform better with smaller chunk sizes, while multi-hop (especially abstract multi-hop) questions might benefit more from larger chunks. So it is hard to tell if the increase in chunk size will increase the performance or not.

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### ✅ Answer #3:

A better embedding model results in more relevant content being retrieved, which helps the LLM to return better answers. The text-embedding-3-large model has higher dimensional vectors that capture semantic meaning more accurately than text-embedding-3-small. This means it can better understand relationships between concepts and find more relevant chunks when performing similarity searches. So in our case, going from text-embedding-3-small to text-embedding-3-large should definitely improve the performance of our application.

In [45]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [46]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [47]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [48]:
dopeness_rag_chain.invoke({"question" : "How are people using AI to make money?"})

"Alright, strap in—the AI money game is more lit than you thought. The context drops some serious knowledge bombs on how ChatGPT and generative AI are flipping the script in the money-making arena.\n\nFirst off, peeps aren’t just using AI to clock in and grind on mundane tasks. Nah, that’s old news. The real sauce is AI acting as a **smart advisor and research assistant**—boosting decision-making, firing up productivity, and unleashing worker output like a productivity beast on steroids.\n\nCollis and Brynjolfsson (2025) put a number on this magic: US users would demand a cool $98 each just to *pause* using generative AI for a month. That’s a jaw-dropping **$97 billion a year in consumer surplus**. Translation? AI’s not just helping people earn—it's creating insane value and economic surplus just by being their confidential sidekick at work and beyond.\n\nSo, the money moves come from:\n- **Augmenting human labor** (not just replacing it)\n- Giving **decision support and research boost

Finally, we can evaluate the new chain on the same test set!

In [49]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'best-pen-78' at:
https://smith.langchain.com/o/54cd3916-cf32-4735-9053-ad02339e1abb/datasets/d9f59057-5dde-4610-b0b5-85fc4c9f7549/compare?selectedSessions=198733f6-c00c-4339-b4e5-34d249a63a8d




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"what July 2025 data show ChatGPT use like, and...","Yo, the July 2025 data on ChatGPT is straight ...",None,"In July 2025, ChatGPT had over 700 million wee...",1,0,1,8.087465,747940ab-46fb-44cd-bae8-27ccf2ff0875,e3ec9c87-5b8a-49c9-88a5-7dec1ca16f56
1,Based on the data indicating that over 700 mil...,"Yo, this data is straight fire! By July 2025, ...",None,"The context shows that by July 2025, ChatGPT h...",1,1,1,4.324843,89fb0173-2ce7-42ad-b075-4a055a5b34ff,502750fe-f1de-47bd-bef3-a7af7427bdcd
2,what Handa et al 2025 say about ChatGPT messag...,"Alright, diving into the vortex of pure data a...",None,Handa et al. (2025) report that nearly 80% of ...,0,1,1,3.784174,e76d8c70-f9a1-4088-81e8-45c2c31b9156,7cc88efe-921e-4d8f-b7e4-ccf4269d6338
3,"Based on the findings of Handa et al. (2025), ...","Alright, let’s blast off with some turbocharge...",None,Handa et al. (2025) report that nearly 80% of ...,1,1,1,7.097683,24f60c41-7ee4-487f-b6de-24977c773987,fd7557f8-7dbc-46a1-8438-011165a82d63
4,Considering the rapid growth of ChatGPT's usag...,"Yo, here’s the rad lowdown straight from the c...",None,"The rapid adoption of ChatGPT, with over 18 bi...",1,1,1,2.955903,e92463db-b06c-434c-86fa-79d6eff672ba,bc4e6fda-5efd-4815-b86d-286c3529c52e
5,How do usage patterns and growth across popula...,"Alright, strap in for the lowdown on ChatGPT’s...",None,The context indicates that ChatGPT's adoption ...,1,1,1,5.841068,de7dcde9-f91f-408f-bc2a-4446a1cf85c0,8ac22a9e-cd0d-490f-8156-1b9b8c00cb57
6,How does the increasing percentage of non-work...,"Yo, here’s the dopest breakdown straight from ...",None,The data shows that non-work messages increase...,1,1,1,6.416976,87e21c66-c293-453d-9ff6-a567b3b32e73,aca9a85a-660c-4e26-9413-2741a6422851
7,Considering the rapid growth of ChatGPT's usag...,"Alright, strap in — let’s ride this AI wave wi...",None,The data indicates that a significant portion ...,1,1,1,6.009318,71939f95-13fe-4731-8c04-1c9c8a2ca6b7,30f4f451-7490-4e79-9fc3-aed3f6072c76
8,What is the significance of Personal Reflectio...,"Alright, let’s slice through the data like a c...",None,"According to the context, Personal Reflection ...",1,1,1,3.982689,2118fe57-7aee-4432-8fb3-06335972010b,32d9cd7a-44bc-4310-bd62-5feeea564177
9,Could you explain the significance of SOC2 cod...,"Alright, let's unpack this with maximum swagge...",None,Variation by Occupation Figure 23 presents var...,1,1,1,4.155683,650a4f17-8cb9-40b1-b9b0-c3af9dc2eeb0,77c20dba-a765-46cb-99bd-36206d116653


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

### Analysis:

The system performed better in Test 2 with the larger chunk size and the better embedding model. Especially for 'Helpfulness,' it performed dramatically better (0.4167 vs. 0.833). For correctness, there was no difference in numbers (both had 10 out of 12 correct).

What did change, though, is that in Test 1 there were two 'I don't know' answers, while in Test 2 the answers were actually wrong. This means the system went from being conservative (saying 'I don't know') in Test 1 to being overconfident (giving wrong answers). This suggests the improvements made the system more likely to find information, but not always the correct information.

However, we changed two parameters at the same time: chunk size and the embedding model. This makes it harder to identify which of these changes had the biggest impact. It was to be expected that the results would be better because of the better embedding model, but we could not see what effect the increase in chunk size had. It could also be possible that the results would be even better with the smaller chunk size and the better embedding model.

That is why I did a third evaluation run with the smaller chunk size but the better model. It turned out that the correctness number did not change. We also had two wrong answers (instead of two non-answers), which suggests that the overconfidence was caused by the better embedding model. What did change was the 'Helpfulness' number - down from 0.833 to 0.75. So the chunk size did have a small impact, but not as much as the better model.

'Dopeness' went from 0 to 1, but this was to be expected. This factor should not affect the result quality because it changes just the tone of the response.

| Test       | Chunk Size | Embedding | Correctness | Helpfulness | Dopeness |
| ---------- | ---------- | --------- | ----------- | ----------- | -------- |
| **Test 1** | 500        | 3-small   | 83%         | 33%         | 0%       |
| **Test 2** | 1000       | 3-large   | 83%         | 83%         | 100%      |
| **Test 3** | 500        | 3-large   | 83%         | 75%         | 100%      |


Here are the tests:

Test 1:

![image](img/langsmith-eval-1.png)

Test 2:

![image](img/langsmith-eval-2.png)

Test 3:

![image](img/langsmith-eval-3.png)
